In [ ]:
from vision.tracker import Tracker, get_marker_object, get_markers, CAM_MAT, CAM_DIST
import cv2
import time

In [ ]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

In [ ]:
vision_thread.start()

In [ ]:
while True:
    robot_m = get_marker_object(7)

    if robot_m.realxy() is not None:
        print(robot_m.realxy()[:2])
    time.sleep(3)